# Download VQAv2 dataset images


In [1]:
#!wget http://images.cocodataset.org/zips/test2015.zip
!mkdir /content/vqav2/

!wget http://images.cocodataset.org/zips/train2014.zip
!wget http://images.cocodataset.org/zips/val2014.zip

!unzip /content/train2014.zip -d /content/vqav2/
!unzip /content/val2014.zip -d /content/vqav2/

The syntax of the command is incorrect.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


# Install custom open_flamingo model with VQAv2 dataset questions and answers


In [ ]:
!rm -rf /content/open_flamingo
!git clone https://github.com/Lavrenti-Mikaelyan/open_flamingo.git

!unzip /content/open_flamingo/vqav2/v2_Questions_Train_mscoco.zip -d /content/vqav2/
!unzip /content/open_flamingo/vqav2/v2_Questions_Val_mscoco.zip -d /content/vqav2/
#!unzip /content/v2_Questions_Test_mscoco.zip -d /content/VQA/Questions

!unzip /content/open_flamingo/vqav2/v2_Annotations_Train_mscoco.zip -d /content/vqav2
!unzip /content/open_flamingo/vqav2/v2_Annotations_Val_mscoco.zip -d /content/vqav2

!pip install scikit-image matplotlib
!pip install -r /content/open_flamingo/requirements.txt
!pip install -r /content/open_flamingo/requirements-eval.txt
from huggingface_hub import hf_hub_download

# Run evaluation on pre-trained Open-Flamingo
Needs 16GB GPU, runs 0, 4, 8 shot experiments
Needs more GPU for 16+ shot
Uses Open Clip Vit-L/14 and MPT-1B-RedPajama-200B
Tests on 4000 VQAv2 samples for each shot config to to limit runtime <2 hrs in total

Prelim results:

  0 shot - 43.53 VQA score

  4 shot - 44.66 VQA score

  8 shot - 44.75 VQA score

  (comparable/close to Open_Flamingo result, can increase sample count to get true accuracy)

This is a Flamingo-3B model. Compared to the performance of the same model in the Flamingo paper, 4-shot VQA is performs about %8 worse. I attribute this to different vision encoder and LLM choices in paper vs Open Flamingo, different pretraining dataset (M3W vs mmc4).

Note, that even the developers of Open Flamingo admit that their models achieve only 80-89% of what the origial Flamingo results were.



In [2]:
import sys
checkpoint_path = hf_hub_download("openflamingo/OpenFlamingo-3B-vitl-mpt1b", "checkpoint.pt", local_dir='/content/open_flamingo', local_dir_use_symlinks=False)
print(checkpoint_path)
sys.path.insert(0,'/content/open_flamingo/open_flamingo')

!chmod +x /content/open_flamingo/open_flamingo/scripts/run_eval.sh
!/content/open_flamingo/open_flamingo/scripts/run_eval.sh

NameError: name 'hf_hub_download' is not defined

In [ ]:
import os
import sys
os.environ['PYTHONFAULTHANDLER'] = '1'
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
os.environ['PYTHONPATH']="/env/python:open_flamingo"

print(os.environ['PYTHONFAULTHANDLER'])
print(os.environ['CUDA_LAUNCH_BLOCKING'])
print(os.environ['PYTHONPATH'])

checkpoint_path = hf_hub_download("openflamingo/OpenFlamingo-3B-vitl-mpt1b", "checkpoint.pt", local_dir='/content/open_flamingo', local_dir_use_symlinks=False)
print(checkpoint_path)
sys.path.insert(0,'/content/open_flamingo/open_flamingo')

!python /content/open_flamingo/open_flamingo/eval/evaluate.py \
    --vision_encoder_path ViT-L-14 \
    --vision_encoder_pretrained openai\
    --lm_path anas-awadalla/mpt-1b-redpajama-200b \
    --lm_tokenizer_path anas-awadalla/mpt-1b-redpajama-200b \
    --cross_attn_every_n_layers 1 \
    --checkpoint_path "/content/open_flamingo/checkpoint.pt" \
    --results_file "results.json" \
    --precision "fp32" \
    --batch_size 16 \
    --shots 4 \
    --num_samples 8000 \
    --eval_vqav2 \
    --vqav2_train_image_dir_path "/content/vqav2/train2014" \
    --vqav2_train_annotations_json_path "/content/vqav2/v2_mscoco_train2014_annotations.json" \
    --vqav2_train_questions_json_path "/content/vqav2/v2_OpenEnded_mscoco_train2014_questions.json" \
    --vqav2_test_image_dir_path "/content/vqav2/val2014" \
    --vqav2_test_annotations_json_path "/content/vqav2/v2_mscoco_val2014_annotations.json" \
    --vqav2_test_questions_json_path "/content/vqav2/v2_OpenEnded_mscoco_val2014_questions.json"